In [1]:
import pandas as pd
import numpy as np

import os
import cx_Oracle

foods = pd.read_csv('./전국통합식품영양성분정보(가공식품)표준데이터.csv', sep=',', encoding='ms949')
foods.head(1)

,식품코드,식품명,데이터구분코드,데이터구분명,식품기원코드,식품기원명,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,유통업체명,수입여부,원산지국코드,원산지국명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자,제공기관코드,제공기관명
0,P109-302030200-2176,과·채주스_프룻밀토마토,P,가공식품,1,가공식품,9,음료류,9302,과·채주스,...,해당없음,N,36.0,해당없음,2,수집,2021-06-30,2024-02-23,1471000,식품의약품안전처


In [2]:
foodSet = pd.DataFrame(foods, columns=['식품코드', '식품명', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '포화지방산(g)', '나트륨(mg)'])

# 결측치 제거  : NaN 있을 경우 K Mean 측정에서 문제 발생
foodSet.fillna(0, inplace=True)

In [3]:
foodSet.head(1)

,식품코드,식품명,에너지(kcal),탄수화물(g),단백질(g),지방(g),포화지방산(g),나트륨(mg)
0,P109-302030200-2176,과·채주스_프룻밀토마토,43,10.0,0.47,0.0,0.0,17.0


In [4]:
# DB 한글 설정에 따른 한글 지원
# os.putenv('NLS_LANG', 'KOREAN_KOREA.KO16MSWIN949');
os.putenv('NLS_LANG', '.UTF8')

conn = cx_Oracle.connect('food', 'food', 'localhost:1521/xe')
cursor = conn.cursor()

In [5]:
# 성별과 나이대로 식약처 권장 영양 정보 조회
def findByGenderAndAge(gender, ageRange):

    sql_str = "select * from nutri_std_tbl where gender=:gender and age=:ageRange"
    records = cursor.execute(sql_str, [gender, ageRange])
    record = records.fetchone()

    # print('레코드 : ', record)

    # ['NUM', 'GENDER', 'AGE', 'CARBOHYDRATE', 'PROTEIN', 
    #  'SUGAR', 'NATRIUM', 'CHOLESTEROL', 'FAT', 'FATTY_ACID', 'TRANS_FATTY_ACID']
   
    return record

In [6]:
# 특정 나이의 나이대를 판정
# ex) 20 => 19~29
def calcAgeBand(age):
		
    result = ""
    
    ageRanges = ("6~8", "9~11", "12~14", "15~18", "19~29", "30~49", "50~64", "65~74", "75~")

    # print(ageRanges)
    
    # 리스트의 나이 최소 나이 추출하여 나이대의 최솟값의 리스트 생성
    # 나이대의 최솟값으로  6,9,12,15,19,30, .....
    ageRangeMins = [int(ageRange.split("~")[0]) for ageRange in ageRanges]
        
    # print(ageRangeMins)
    
    # 해당되는 나이(가령 20) 보다 이하 인(작거나 같은) 나이 중에서 최댓값을 구함 => 19
    tempAge = max([ageRangeMin for ageRangeMin in ageRangeMins if ageRangeMin <= age])
    
    # print("tempAge :", tempAge)
    
    # 나이대 계산
    result = [ageRange for ageRange in ageRanges if str(tempAge) in ageRange]

    # print(result[0])
        
    return result[0]

# 성별과 나이대로 식약처 권장 영양 정보 조회
def findByGenderAndAge(gender, ageRange):

    sql_str = "select * from nutri_std_tbl where gender=:gender and age=:ageRange"
    records = cursor.execute(sql_str, [gender, ageRange])
    record = records.fetchone()

    # print('레코드 : ', record)

    # ['NUM', 'GENDER', 'AGE', 'ENERGY', 'CARBOHYDRATE', 'PROTEIN', 
    #  'SUGAR', 'NATRIUM', 'CHOLESTEROL', 'FAT', 'FATTY_ACID', 'TRANS_FATTY_ACID']
   
    return record

findByGenderAndAge('남', calcAgeBand(20))
# (5,  '남', '19~29',   2600,         130,          65,     '10~20', 1500,        300,       '15~30',          7,            1)
# 번호, 성별, 연령대, 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)

(5, '남', '19~29', 2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1)

In [7]:
# 성별, 나이, 신장, 체중, PA(Physical Activity:신체활동량)
# 영양권장량 (Recommended Dietary Allowance, RDA)

# 에너지 요구량 추정치(ESTIMATED ENERGY REQUIREMENT (EER))

# 관련 논문(근거) : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/
# https://globalrph.com/medcalcs/estimated-energy-requirement-eer-equation/
# https://www.diabetes.or.kr/general/dietary/dietary_02.php?con=2
# https://www.eatforhealth.gov.au/nutrition-calculators/daily-energy-requirements-calculator

# PA(남성) : 1.1(저활동적), 1.48(매우 활동적)
# 원문) PA = 1.0 (sedentary), 1.11 (low active), 1.25 (active), or 1.48 (very active).

# PA(여성) : 1.2(저활동적), 1.45(매우 활동적)
# 원문) PA = 1.0 (sedentary), 1.12 (low active), 1.27 (active), or 1.45 (very active).

# PA 기준안 그림 포함)
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/bin/JENB_2019_v23n4_1_f001.jpg

# 한국인 영양섭취 기준 : https://www.mohw.go.kr/boardDownload.es?bid=0019&list_no=335863&seq=1

# 영양권장량(RDA) 계산
# weight(체중) : kg, 
# height(신장) : m
# EER(에너지 요구량 추정치(ESTIMATED ENERGY REQUIREMENT (EER))) : kcal
# disease(질병) : 고혈압, 당뇨, 고지혈증, 비만
def calculateRDA(disease, gender, age, height, weight, PA=1):

    #recom = [] # 권장량

    # 에너지 추정량(kcal) : ESTIMATED ENERGY REQUIREMENT (EER) EQUATION
    # 관련 논문(근거) : https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7004509/
    # https://globalrph.com/medcalcs/estimated-energy-requirement-eer-equation/
    EER = 0

    if gender == '남':
        EER = 622 - 9.53 * age + PA * (15.91 * weight + 536.6 * height)
    elif gender == '여':    
        EER = 354 - 6.91 * age + PA * (9.36 * weight + 726 * height)

    # 식약처 기준량 조회
    # 번호, 성별, 연령대, 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)
    age_arange = calcAgeBand(age)
    print("연령대 : ", age_arange)
    
    nutri_tot = findByGenderAndAge(gender, age_arange)
    nutri_recomm = list(nutri_tot)
    
    # 불필요성분 제거 ex) 번호, 성별, 연령대
    nutri_recomm = nutri_recomm[3:]
    
    # 권장열량(kcal), 탄수화물(g), 단백질(g), 지방(g), 당류(g),  나트륨(mg), 콜레스테롤(mg),  포화지방산(g), 트랜스지방(g)
    print("권장 섭취량 (일반) : ", nutri_recomm)

    # 관련 근거 : https://www.ncbi.nlm.nih.gov/books/NBK218738/

    # 참고) 연도별 우리 국민의 1일 나트륨평균 섭취량 추이
    # (2022.12.31. 기준, 단위: ㎎/day, 출처: 국민건강영양조사, 질병관리청)
    
    # 연도           2011 2012  2013  2014  2015  2016  2017  2018  2019  2020  2021
    # 나트륨 섭취량 4,831 4,583 4,027 3,890 3,890 3,669 3,478 3,274 3,289 3,220 3,080

    # 관련 근거 : 2020 한국인 영양소 섭취기준 활용 자료 개발
    # https://e-jnh.org/pdf/10.4163/jnh.2022.55.1.21
    
    # 질병이 있을 경우  ex) disease(질병) : 고혈압, 당뇨, 고지혈증, 비만

    # 참고) 고혈압의 경우 나트륨 권장량이 3700ml로 되어 있는 것은 권장량과 다르나 최근 식약처의 충분량 섭취 제개정 기준 변화로 그대로 수용
    # 나트륨 관련 규정 : 2020 한국인 영양소 섭취기준 <보건복지부, 한국영양학회> 1. 한국인 영양소 섭취기준 제·개정 요약 p. 27
    # https://www.kns.or.kr/FileRoom/FileRoom_view.asp?idx=108&BoardID=Kdr
    
    # "당뇨 환자의 탄수화물 섭뤼량이 많은 이유 !"
    # 당뇨에 따른 탄수화물 섭취량 관련 논문)
    #
    # "당뇨병 환자는 일반적으로 총 열량의 50~60%를 탄수화물로 섭취하도록 권고하고 있으며, 탄수화물, 단백질, 
    # 지방 섭취량은 식습관, 기호도, 치료목표 등을 고려하여 개별화할 수 있다"
    # 논문 인용) 탄수화물계산을 활용한 임상영양요법 : 김미라 분당서울대학교병원 영양실
    # : https://www.e-jkd.org/upload/pdf/jkd-2022-23-2-133.pdf

    # 참고) 당뇨 환자 일일 필요 열량 측정 프로그램 : https://www.diabetes.or.kr/general/dietary/dietary_02.php?con=2

    if (disease == '고혈압'):
        nutri_recomm[5] = 3700 # 단위 mg
        print("nutri_recomm[5] (나트륨) : ", nutri_recomm[5])
    
    if (disease == '당뇨'):
        nutri_recomm[1] = EER * 0.7 // 4 # 단위 g        
        print("nutri_recomm[1] (탄수화물) : ", nutri_recomm[1]) # 탄수화물
    
    if (disease == '고지혈증'):
        nutri_recomm[6] = EER * 0.07 // 9 # 단위 g
        print("nutri_recomm[6] (포화지방산) : ", nutri_recomm[6]) # 포화지방산

    print("권장 섭취량(유병자 적용 후) : ", nutri_recomm)

    return nutri_recomm

In [8]:
calculateRDA('', '남', 20, 1.7, 70, 1.25) 

연령대 :  19~29
권장 섭취량 (일반) :  [2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1]
권장 섭취량(유병자 적용 후) :  [2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1]


[2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1]

In [9]:
calculateRDA('고혈압', '남', 20, 1.7, 70, 1.25) 

연령대 :  19~29
권장 섭취량 (일반) :  [2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1]
nutri_recomm[5] (나트륨) :  3700
권장 섭취량(유병자 적용 후) :  [2600, 130, 65, '10~20', 1500, 3700, '15~30', 7, 1]


[2600, 130, 65, '10~20', 1500, 3700, '15~30', 7, 1]

In [10]:
calculateRDA('당뇨', '남', 20, 1.7, 70, 1.25) 

연령대 :  19~29
권장 섭취량 (일반) :  [2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1]
nutri_recomm[1] (탄수화물) :  518.0
권장 섭취량(유병자 적용 후) :  [2600, 518.0, 65, '10~20', 1500, 300, '15~30', 7, 1]


[2600, 518.0, 65, '10~20', 1500, 300, '15~30', 7, 1]

In [11]:
calculateRDA('고지혈증', '남', 20, 1.7, 70, 1.25) 

연령대 :  19~29
권장 섭취량 (일반) :  [2600, 130, 65, '10~20', 1500, 300, '15~30', 7, 1]
nutri_recomm[6] (포화지방산) :  23.0
권장 섭취량(유병자 적용 후) :  [2600, 130, 65, '10~20', 1500, 300, 23.0, 7, 1]


[2600, 130, 65, '10~20', 1500, 300, 23.0, 7, 1]

---

In [15]:
# 특정 음식에 대한 군집 내의 유사 대체품 검색 : 최단거리 탐색
# 군집화된 csv 로딩

foodSet = pd.read_csv('./Final_Cluster_Result.csv')
foodSet.head(3)

,Unnamed: 0,식품코드,식품명,에너지(kcal),탄수화물(g),단백질(g),지방(g),포화지방산(g),나트륨(mg),cluster,clusterLabel
0,0,P109-302030200-2176,과·채주스_프룻밀토마토,43,10.0,0.47,0.0,0.0,17.0,2,1
1,1,P109-302030200-2177,과·채주스_프리마5후르츠칵테일주스,45,10.7,0.30,0.0,0.0,0.0,2,1
2,2,P109-302030200-2178,과·채주스_프리마셀렉션사과주스,45,11.0,0.20,0.2,0.0,0.0,2,1


In [40]:
# 입력 음식(오렌지주스)에 대한 정보 조회

# test
# foodSet.loc[foodSet['식품명'] == '과·채주스_프리마셀렉션사과주스']

# foodSet['식품명'] == '오렌지주스'
# 단어 포함 유사검색으로 조회
# 다수일 경우 첫번째 행 추출

inputFood = foodSet.loc[foodSet['식품명'].str.contains('오렌지주스')].iloc[0]


Unnamed: 0                      130
식품코드            P109-302030200-2068
식품명             과·채주스_파스퇴르발렌시아오렌지주스
에너지(kcal)                        43
탄수화물(g)                        10.0
단백질(g)                          0.4
지방(g)                           0.1
포화지방산(g)                       0.05
나트륨(mg)                        10.0
cluster                           2
clusterLabel                      1
Name: 130, dtype: object

In [12]:
cursor.close()
conn.close()